# Example: Class with `Streamer` method overridden

## Class definition in `ROOT`

### TOverrideStreamer

```cpp
class TOverrideStreamer : public TObject {
    int m_int{ 0 };
    double m_double{ 0.0 };

    // ... other codes ...
};
```

We override its `Streamer` method, which reads/writes data with an additional tag:

```cpp
void TOverrideStreamer::Streamer( TBuffer& b ) {
    if ( b.IsWriting() )
    {
        TObject::Streamer( b ); // Call base class Streamer
        b << m_int;
        b << 0x12345678; // Write a tag
        b << m_double;
    }
    else
    {
        // ... reading logic ...
    }
}
```

### TObjInSTL

```cpp
class TObjInSTL : public TObject {
    std::vector<TOverrideStreamer> m_objVec;
    std::map<int, TOverrideStreamer> m_objMap;

  // ... other codes ...
};
```

## `uproot-custom` Implementation

To read `TOverrideStreamer`, several steps are performed.

### Step 1: Python factory instantiation

Data is tree-like, so Python factory needs to match specific nodes to process.

```python
class OverrideStreamerFactory(Factory):
    @classmethod
    def build_factory(
        cls,
        top_type_name: str,
        cur_streamer_info: dict,
        all_streamer_info: dict,
        item_path: str,
        **kwargs,
    ):
        fName = cur_streamer_info["fName"]
        if fName != "TOverrideStreamer" and top_type_name != "TOverrideStreamer":
            return None

        return cls(fName)
```

### Step 2: Build C++ reader

The C++ reader

1. Reads binary data according to the overridden `Streamer` method.

    ```cpp
    void read( BinaryBuffer& buffer ) {
        buffer.skip_TObject();

        m_data_ints->push_back( buffer.read<int>() );

        auto tag = buffer.read<uint32_t>();
        if ( tag != 0x12345678 ) throw std::runtime_error( "Error: Unexpected tag value." );

        m_data_doubles->push_back( buffer.read<double>() );
    }
    ```

2. Returns the read data to Python.

    ```cpp
    py::object data() const {
        auto int_array    = make_array( m_data_ints );
        auto double_array = make_array( m_data_doubles );
        return py::make_tuple( int_array, double_array );
    }
    ```

The Python factory

3. Transforms the returned data into an `awkward` array.

    ```python
    def make_awkward_content(self, raw_data):
        int_array, double_array = raw_data

        return awkward.contents.RecordArray(
            [
                awkward.contents.NumpyArray(int_array),
                awkward.contents.NumpyArray(double_array),
            ],
            ["m_int", "m_double"],
        )
    ```

### C++ reader

```cpp
/*
* Reads data from the BinaryBuffer, applying custom logic to handle
* overridden streamers.
*/
void read( BinaryBuffer& buffer ) {
    buffer.skip_TObject();

    m_data_ints->push_back( buffer.read<int>() );

    auto tag = buffer.read<uint32_t>();
    if ( tag != 0x12345678 ) throw std::runtime_error( "Error: Unexpected tag value." );

    m_data_doubles->push_back( buffer.read<double>() );
}
```

### Directly read from TBranch

We define another class that contains STL containers of the above class:

```cpp
class TObjInSTL : public TObject {
private:
  std::vector<TOverrideStreamer> m_objVec;
  std::map<int, TOverrideStreamer> m_objMap;

  // ... other codes ...
};
```